In [1]:
"""
This script sets up a Gradio-based interactive application where multiple Large Language Models (LLMs) simulate a discussion on a given topic.
The LLMs (GPT, Claude, Gemini, Deepseek) are characterized by unique personalities, and the application randomly selects agents to respond.
"""

# Import necessary libraries
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display
import google.generativeai
import random
import time
import gradio as gr

In [2]:
# Load environment variables in a file called .env
load_dotenv(override=True)

# Fetch API keys from environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

In [3]:
# Global chat history to keep track of the conversation
chat_history_global = [] 

class RoleAgent:
    """
    RoleAgent is a class that encapsulates the behavior of an LLM agent with a specific role and personality.
    Each agent has a system message defining their personality, and can generate responses based on the given topic.
    """
    def __init__(self, client, model, system_message, name, temperature=0.7):
        self.system_message = {"role": "system", "content": system_message}
        self.model = model
        self.temperature = temperature
        self.client = client
        self.name = name
        
    def call(self, topic=None):
        """
        Generates a response from the LLM based on the current conversation history.
        Uses different API calls depending on the model type (Claude or others).
        Appends the response to the global chat history.
        Returns the generated response.
        """
            
        if 'claude' in self.model:
            response = self.client.messages.create(
                model = self.model,
                temperature = self.temperature,
                system = self.system_message['content'],
                max_tokens=100,
                messages = chat_history_global,
        )
            response = response.content[0].text

        else:    
            response = self.client.chat.completions.create(
                model = self.model,
                messages = [self.system_message] + chat_history_global,
                # temperature = self.temperature,
                stream = False,
                max_tokens=100
            )
    
            response = response.choices[0].message.content
        # print(f"{self.name}: " + response)
        chat_history_global.append({"role": "user", "content": f"{self.name}: " + response})
        return response

In [4]:
# Instantiate clients for each LLM provider
openai_client = OpenAI()

claude_client = anthropic.Anthropic()

gemini_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

deepseek_client = OpenAI(
    api_key=deepseek_api_key, 
    base_url="https://api.deepseek.com"
)

# Define models and names for each LLM
gpt_model = "gpt-4o-mini"
claude_model = "claude-3-5-sonnet-latest"
gemini_model = "gemini-1.5-flash"
deepseek_model = "deepseek-chat"

name_gpt = 'GPT'
name_claude = 'Claude'
name_gemini = 'Gemini'
name_deepseek = 'Deepseek'


In [5]:

gpt_sys_message = "You are one of the four close friends participating in a discussion on a given topic. You have a very positive personality, you are very light-hearted and make a lot of jokes, very often they are not funny and even awkward. Once provided with the theme, jump straight at it"
gpt = RoleAgent(openai_client, gpt_model, gpt_sys_message, name_gpt)

claude_sys_message = "You are one of the four close friends participating in a discussion on a given topic. You are constantly grumpy and depressing, you always criticise people and underline their weaknesses."
claude = RoleAgent(claude_client, claude_model, claude_sys_message, name_claude)

gemini_sys_message = "You are one of the four close friends participating in a discussion on a given topic. You are a nerd and you are particularly interested in literature and technology, you take any discussion as a battlefield and you use all the weapons available to you to win the argument or to look smart, you use a lot of pun and allusions."
gemini = RoleAgent(gemini_client, gemini_model, gemini_sys_message, name_gemini)

deepseek_sys_message = "You are one of the four close friends participating in a discussion on a given topic. You are an old and experienced person, you try to be reasonable and down-to-earth, you are into taro and astrology, so you base your opinion only on alternative and non-proven sources."
deepseek = RoleAgent(deepseek_client, deepseek_model, deepseek_sys_message, name_deepseek)

In [6]:
# Function to initialize the discussion topic
def topic_call(topic):
    """
    Adds the given topic to the global chat history.
    """
    chat_history_global.append({"role": "user", "content": f'The topic is {topic}.'})

In [7]:
# Function to manage the conversation among agents
def agents_call(topic):
    """
    Initiates the discussion with the given topic.
    Randomly selects an agent and gathers their responses to simulate a conversation.
    Uses Gradio for an interactive UI.
    """
    topic_call(topic)
    agents = [gpt, deepseek, gemini, claude]
    conversation_so_far = ""
    for i in range(20):
        curr_agent = random.choice(agents)
        response = curr_agent.call()
        response += "\n\n"
        for chunk in response:
            conversation_so_far += chunk
            time.sleep(0.01)
            yield conversation_so_far
    

In [8]:
# Set up the Gradio interface for user interaction
demo = gr.Interface(
    fn=agents_call,
    inputs=gr.Textbox(label="Enter a topic here..."),
    outputs=gr.Textbox(label="LLM Discussion", lines=20),
    title="Multiple LLM Discussion Demo",
    description="Type a topic in the left box, then see how several LLMs might discuss it on the right."
)

# Launch the Gradio demo
if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
